# Лабораторная работа 2
# Тема 2. Генерация простых чисел

# <font color='red'>Разбираю теоретическую  часть</font>

## Тестирование чисел на простоту

### Метод пробных делений

#### Алгоритм

**Метод пробных делений.**  
Input: $n\in\mathbb{N}.$  
Output: $n$ -- простое?    
1$.\;\; d:=2$  
2$.\;$ While $\;d\leq \sqrt n$   
$\qquad$ If $\;n = 0 \mod d$  
$\qquad\qquad$ Return False   
$\qquad$  $d:=d+1$    
3$.\;$ Return True

#### Реализация

In [88]:
def TrialDivision(n):
    # counter = 0
    d = 2
    while d <= n**0.5:
        if n % d == 0:
            return False
        d += 1
        # counter += 1
    # return True, counter
    return True

n = 43
TrialDivision(n)

True

#### Кол-во арифметических операций и временная сложность

Если $n$ -- простое, то количество арифметических операций, выполняемых во время работы алгоритма, равно  $$f(n)=\left[\sqrt n\right]-1=O\left(n^{1/2}\right).$$

Поэтому для временной сложности алгоритма справедлива оценка
$$
  T(N):=\max\limits_{\left<n\right>\leq N} f(n)=O\left(2^{N/2}\right).
$$
Метод пробных делений прост для понимания и реализации, но экспоненциален.

#### Замечания

Вопрос: почему количество операций будет $\left[\sqrt n\right] - 1?$

Ответ: потому что проверка на простоту начинается не с единицы, а с двойки $d := 2$.

Вопрос: почему алгоритм должен работать пока $d \le \sqrt n?$

Ответ: потому что после $\sqrt n$ будут встречаться те же самые множители числа $n$, что и до корня. А если множителей не нашлось, то и после корна их тем более не будет.

### Проверка теорем

#### Кольцо вычетов из предыдущей лабораторной (но с ошибками при делении и разности)

In [89]:
class Ring:
    def __init__(self, remainder, m):
        self.remainder = remainder
        self.mod = m

    def __repr__(self):
        return "(%s)" % (self.remainder)
    
    def __add__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder + other.remainder) % self.mod, self.mod)
        
    def __sub__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder - other.remainder) % self.mod, self.mod)
        
    def __mul__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder * other.remainder) % self.mod, self.mod)
        
    def __truediv__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder / other.remainder) % self.mod, self.mod)
        
    def __pow__(self, num):
        return Ring(NewFastPower(self.remainder, num, self.mod), self.mod)
        
    def __neg__(self):
        return Ring(reverse_elem(self.remainder, self.mod), self.mod)
        
    def __eq__(self, other):
        if isinstance(other, Ring):
            if self.remainder == other.remainder:
                return True
            else:
                return False
            
def reverse_elem(a, b):
    if a == b - 1:
        return a % b
    a_copy = a
    b_copy = b
    m11, m12 = 1, 0
    m21, m22 = 0, 1
    while b:
        q = a // b
        a, b = b, a % b
        m11, m12 = m12, m11 - m12*q
        m21, m22 = m22, m21 - m22*q
    gcd = m11 * a_copy + m21 * b_copy
    if gcd == 1:
        return m11
    else:
        raise ValueError('base is not invertible for the given modulus')
        
# если я учитываю, что a - целое и, что есть проблема с единицей, то
def NewFastPower(a, d, m):
    b = abs(a)
    if d == 1:
        return b % m
    bin_lst = [int(i) for i in bin(d)[3:]]
    for elem in bin_lst:
        b = (b ** 2) * (a ** elem) % m
    return b

#### Проверяю выполнение теоремы Эйлера на конкретном примере

##### Мультипликативная група кольца вычетов по модулю n

Пример из википедии: https://ru.wikipedia.org/wiki/%D0%9C%D1%83%D0%BB%D1%8C%D1%82%D0%B8%D0%BF%D0%BB%D0%B8%D0%BA%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D0%B3%D1%80%D1%83%D0%BF%D0%BF%D0%B0_%D0%BA%D0%BE%D0%BB%D1%8C%D1%86%D0%B0_%D0%B2%D1%8B%D1%87%D0%B5%D1%82%D0%BE%D0%B2

In [90]:
n = 42
Z_star_n = [Ring(1, 42), Ring(5, 42), Ring(11, 42), Ring(13, 42), Ring(17, 42), Ring(19, 42), Ring(23, 42), Ring(25, 42), Ring(29, 42), Ring(31, 42), Ring(37, 42), Ring(41, 42)]
Z_star_n

[(1), (5), (11), (13), (17), (19), (23), (25), (29), (31), (37), (41)]

##### Выполнение теоремы Эйлера

**Теорема Эйлера**  
 Пусть $n$ натуральное число, не равное $1.$ Если
   ${a\in\mathbb{Z}_n^*,}$ то
    $$
      {a^{\varphi(n)}\equiv1\mod n.}
    $$

In [91]:
import sympy # для использования реализации функйии Эйлера sympy.totient(n)
import random
main_natural_num = 100
n1 = random.randint(1, main_natural_num) # случайное натуральное число от 1 до 100
a = Ring((Z_star_n[random.randint(0, len(Z_star_n)) - 1]).remainder, n1) # случайный элемент из мультипликативной группы, но по модулю n1
print(n1, a)
((a ** sympy.totient(n1)).remainder) == (1 % n1)

59 (41)


True

#### Проверяю выполнение малой теоремы Ферма

##### Выполнение малой теоремы Ферма

**Малая теорема Ферма**  
Пусть $p$ -- простое число. Тогда для любого числа $a$
    такого, что $0<a<p,$ справедливо сравнение
    $$
      a^{p-1}\equiv 1\mod p.
    $$

In [95]:
p = 11
e = 1e-10
a1 = random.randint(1, p - 1) # случайное натуральное число от из промежутка (0, p)
print(p, a1)
(NewFastPower(a1, p - 1, p)) == (1 % p)

11 4


True

# <font color='red'>Задание 1.</font>

Являются ли числа 
7957, 8321, 13747, 18721, 19951
псевдопростыми по основанию 2?

## Материалы из лекции

Нечётное составное число $n$ называется <font color='blue'>псевдопростым по основанию $a,$</font> $1<a<n,$ если  
 $$
 a^{n-1}\equiv1\mod n.
 $$

Наименьшим псевдопростым числом по основанию 2 является число $341=11\times
     31.$ 
     
А по основанию 3 -- число 91. 

Наименьшим псевдопростым числом является первое нечётное составное число 9, оно псевдопростое по основанию 8
     $$
       8^{8}\equiv((-1)^{2})^4\equiv1^4\equiv1\mod 9.
     $$

In [96]:
(NewFastPower(2, 340, 341)) == (1 % p)

True

## Решение

In [97]:
nums = (7957, 8321, 13747, 18721, 19951)
for el in nums:
    print(NewFastPower(2, el - 1, el))

1
1
1
1
1


Вывод: все числа из кортежа nums являются псевдопростыми.

# <font color='red'>Задание 2.</font>

Найти число Кармайкла, порядковый номер которого соответствует Вашему порядковому номеру в списке подгруппы.

## Материалы из лекции

### Числа Кармайкла

**Определение.**  
Составное число $n\in\mathbb{N}$ называется <font color='blue'>числом Кармайкла</font>,     если для всех натуральных $a$  таких, что $1<a<n,$ $НОД(a,n)=1,$ справедливо     сравнение 
$$
  a^{n-1}\equiv1\mod n.
$$
    
**Пример.**  
 Наименьшим является 561,    далее идут 1105, 1729, 2465.
 
 Чисел Кармайкла бесконечно много.

## Вспомогательная функция

In [98]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

## Решение

In [120]:
def get_Carmichael_numbers(order):
    n = 561
    count = 0
    while count != order:
        gcd_lst = []
        flag = True
        for a in range(2, n):
            if gcd(a, n) == 1:
                gcd_lst += [a]
        for el in gcd_lst:
            if (NewFastPower(el, n - 1, n) != 1):
                flag = False
                break
        if flag:
            yield n
            count += 1
        n += 1

In [100]:
gen_obj = get_Carmichael_numbers(5)

In [101]:
[x for x in gen_obj]

[561, 563, 569, 571, 577]

# <font color='red'>Задание 3.</font>

Оценить количество арифметических операций в Тесте Миллера-Рабина. Оценить временную сложность алгоритма. Является ли он полиномиальным?

# <font color='red'>Задание 4.</font>

Используя Тесте Миллера-Рабина, написать функцию, которая генерирует случайное простое число, состоящее из $b$ бит. Проверить полученное число при помощи функции isprime(p) модуля sympy.

## Решение

In [154]:
def get_random_bits(b):
    bin_num = [0] * b
    bin_num[0] = 1
    bin_num[-1] = 1
    for i in range(1, b):
        bin_num[i] = random.randint(0, 1)
    return bin_num

In [155]:
get_random_bits(6)

[1, 1, 1, 0, 1, 1]

In [184]:
def get_num_from_bin(bin_num):
    num = 0
    i = len(bin_num) - 1
    for bit in bin_num:
        num +=  bit * (2 ** i)
        i -= 1
    return num

In [185]:
n = get_random_bits(6)
print(n)
get_num_from_bin(n)

[1, 0, 1, 1, 1, 1]


47

In [179]:
def st(n):
    s = 0
    t = n - 1
    while t % 2 == 0:
        s += 1
        t = t // 2
    return s, t
st(9)

(3, 1)

In [180]:
def WitnessQ(a, n, s=0, t=0):
    if n % 2 == 0:
        return False
    if s == 0:
        s, t = st(n)    
    if gcd(a,n) > 1:
        return False
    b = pow(a,t,n)
    if b == 1 or b == n-1:
        return True
    for _ in range(1,s):
        b = b**2 % n
        if b == n-1:
            return True
    return False
print(WitnessQ(2,7))
print(WitnessQ(2,9))
print(WitnessQ(8,9))

True
False
True


In [189]:
from random import randint
def MillerRabinTest(n, k = 10):
    s,t = st(n)
    for _ in range(k):
        a = randint(2, n-1)
        if not WitnessQ(a, n, s, t):
            return False
    return True

In [190]:
def gen_Miller_Rabin_num(b, k = 10):
    num = 0
    while True:
        bin_num = get_random_bits(b)
        num = get_num_from_bin(bin_num)
        s, t = st(num)
        if MillerRabinTest(num, k):
            break
    return num

In [204]:
pr_num = gen_Miller_Rabin_num(100)
print(pr_num)
from sympy import isprime
isprime(pr_num)

1217156702870409496771330653089


True